<a href="https://colab.research.google.com/github/sagar9926/EPAI_session16/blob/main/EPAI_session_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 1 :

The files are:

- personal_info.csv - personal information such as name, gender, etc. (one row per person)

- vehicles.csv - what vehicle people own (one row per person)

- employment.csv - where a person is employed (one row per person)

- update_status.csv - when the person's data was created and last updated

Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

In [1]:
!git clone https://github.com/sagar9926/EPAI_session16.git

Cloning into 'EPAI_session16'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 48 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [2]:
cd EPAI_session16

/content/EPAI_session16


In [3]:
!ls

Cars.csv	EPAI_session_16.ipynb  README.md	  vehicles.csv
employment.csv	personal_info.csv      update_status.csv


In [4]:
from collections import namedtuple
import datetime
import csv

In [5]:
def read_file(file_name):
  with open(file_name) as f:
    rows = csv.reader(f,delimiter = ',',quotechar = '"')
    yield from rows


In [6]:
data_type_dict = {
    "employment.csv" : [str,str,str,str],
    'personal_info.csv' : [str,str,str,str,str],
    'update_status.csv' : [str,datetime.datetime.strptime,datetime.datetime.strptime],
    'vehicles.csv' : [str,	str,	str,	int]

}
date_format = '%Y-%m-%dT%H:%M:%SZ'

In [7]:
class file_iter:
  def __init__(self,file_name):
    self.file_name = file_name
    self.data_type = data_type_dict[self.file_name]
    self.file = read_file(self.file_name)
    self.columns = " ".join(next(self.file))
    self.row_name_tuple = namedtuple((self.file_name[0].upper() + self.file_name[1:-4]),self.columns)
    self.row_index = 0
  
  def __iter__(self):
    for row in self.file:
      if self.file_name != 'update_status.csv':
        row = [type_(data) for type_ , data in zip(self.data_type,row)]
      else :
        row = [type_(data) if type_ == str else type_(data,date_format) for type_ , data in zip(self.data_type,row)]
      yield self.row_name_tuple(*row)



### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

In [8]:
employment_iter = file_iter('employment.csv')
personal_info_iter = file_iter('personal_info.csv')
update_status_iter = file_iter('update_status.csv')
vehicles_iter = file_iter('vehicles.csv')

In [9]:
count = 0
for x in personal_info_iter: 
  print(x)
  count += 1
  if count == 3:
    break

Personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Personal_info(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Personal_info(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')


In [10]:
count = 0
for x in update_status_iter: 
  print(x)
  count += 1
  if count == 3:
    break

Update_status(ssn='100-53-9824', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
Update_status(ssn='101-71-4702', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57))
Update_status(ssn='101-84-0356', last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), created=datetime.datetime(2016, 9, 21, 23, 4, 7))


In [11]:
count = 0
for x in vehicles_iter: 
  print(x)
  count += 1
  if count == 3:
    break

Vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
Vehicles(ssn='101-71-4702', vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
Vehicles(ssn='101-84-0356', vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)


In [12]:
qc = file_iter('employment.csv')

In [13]:
qc

In [14]:
next(iter(qc))

Employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')

### Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns. Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

In [15]:
employment_iter = file_iter('employment.csv')
personal_info_iter = file_iter('personal_info.csv')
update_status_iter = file_iter('update_status.csv')
vehicles_iter = file_iter('vehicles.csv')

In [16]:
class AllData :
  def __init__(self,employment,personal_info,update_status,vehicles):
    self.employment_fname = employment
    self.personal_info_fname = personal_info
    self.update_status_fname = update_status
    self.vehicles_fname = vehicles

  def __iter__(self):
    return self.AllDataIterator(self.employment_fname,self.personal_info_fname,self.update_status_fname,self.vehicles_fname)

  class AllDataIterator:
    def __init__(self,employment,personal_info,update_status,vehicles):
      self.employment_iter = file_iter(employment)
      self.personal_info_iter = file_iter(personal_info)
      self.update_status_iter = file_iter(update_status)
      self.vehicles_iter = file_iter(vehicles)
      self.row_index = 0

    def __iter__(self):
      return self

    def __next__(self):
      if self.row_index >= 1000:
        raise StopIteration
      else:
        self.row_index += 1
        for employment , personal_info , update_status , vehicles in zip(self.employment_iter,self.personal_info_iter,self.update_status_iter,self.vehicles_iter):
          employment_dict = dict(employment._asdict()) 
          personal_info_dict = dict(personal_info._asdict())
          update_status_dict = dict(update_status._asdict())
          vehicles_dict = dict(vehicles._asdict())

          combined_dict = {**employment_dict , **personal_info_dict , **update_status_dict , **vehicles_dict }
          ALL_Data = namedtuple('ALL_Data',combined_dict)

          return ALL_Data(**combined_dict)
          
      

In [17]:
qc = AllData('employment.csv','personal_info.csv','update_status.csv','vehicles.csv')

In [18]:
next(iter(qc))

ALL_Data(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)

### Goal 3
Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the last_updated field from the status_update file.

In [19]:
def Stale_records():
  data = AllData('employment.csv','personal_info.csv','update_status.csv','vehicles.csv')
  for NT in data:
    if NT.last_updated < datetime.datetime.strptime("2017/03/01","%Y/%m/%d"):
      yield NT
    else :
      continue

### Goal 4
Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [20]:
data = AllData('employment.csv','personal_info.csv','update_status.csv','vehicles.csv')
car_make = {k.vehicle_make : 0 for k in data}

gender_car_make = {"Male" : car_make.copy(), "Female" : car_make.copy()}


In [21]:
for row in data:
  gender_car_make[row.gender][row.vehicle_make] += 1

In [22]:
sorted(gender_car_make['Female'].items(), key = lambda x : x[1],reverse = True)[:3]

[('Ford', 48), ('Chevrolet', 48), ('Mitsubishi', 25)]

In [23]:
sorted(gender_car_make['Male'].items(), key = lambda x : x[1],reverse = True)[:3]

[('Ford', 44), ('Chevrolet', 38), ('GMC', 31)]

In [24]:
print("Largest number of car makes for Males : ", sorted(gender_car_make['Male'].items(), key = lambda x : x[1],reverse = True)[:1])
print("Largest number of car makes for Females : ", sorted(gender_car_make['Female'].items(), key = lambda x : x[1],reverse = True)[:2])

Largest number of car makes for Males :  [('Ford', 44)]
Largest number of car makes for Females :  [('Ford', 48), ('Chevrolet', 48)]


## Assignment 2 :

### Goal 1
For this goal, you are given a number of CSV files, each of which have their first row with the field name.

You goal is to create a context manager that you can use to produce the data from each file in a named tuple with field names corresponding to the header row field names.

You should use the csv module's reader function to help with parsing the data.

Your context manager should be generic in the sense that it should just need the file name, no other configuration or hardcoded functionality is required. You do not need to worry about data types for this goal - just return every field as a string.

In addition, your context manager should produce lazy iterators.

Implement this using a class that implements the context manager protocol

In [25]:
def dialect_finder(file_name):
  with open(file_name) as f:
      sample = f.read(2000)
      dialect = csv.Sniffer().sniff(sample)
  return(dialect)

In [26]:
def read_file(file_name):
  dialect = dialect_finder(file_name)
  with open(file_name) as f:
    rows = csv.reader(f,dialect)
    yield from rows

In [27]:
class File_Context_Manager :
  def __init__(self,fname):
    self._fname = fname

  def __iter__(self):
    return self

  def __next__(self):
    for data in self.file:
      return(self.data_NT(*data))

  def __enter__(self):
    print(f"Opening File : {self._fname}")
    self.file = read_file(self._fname)
    columns = " ".join(next(self.file))
    self.data_NT = namedtuple((self._fname[0].upper() + self._fname[1:-4]), columns)
    return self

  def __exit__(self,exc_type,exc_value,exc_traceback):
    print(f"Closing File : {self._fname} ")
    self.file.close()
    return False


In [28]:
with File_Context_Manager("personal_info.csv") as f:
  print(next(f))
  print(next(f))
  print(next(f))
  print(next(f))
  print(next(f))

Opening File : personal_info.csv
Personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Personal_info(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Personal_info(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
Personal_info(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
Personal_info(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
Closing File : personal_info.csv 


In [29]:
with File_Context_Manager("Cars.csv") as f:
  print(next(f))
  print(next(f))
  print(next(f))
  print(next(f))
  print(next(f))

Opening File : Cars.csv
Cars(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Cars(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Cars(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
Cars(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration='12.0', Model='70', Origin='US')
Cars(Car='Ford Torino', MPG='17.0', Cylinders='8', Displacement='302.0', Horsepower='140.0', Weight='3449.', Acceleration='10.5', Model='70', Origin='US')
Closing File : Cars.csv 


### Goal 2
The goal is to reproduce the work you did in Goal 1, but using a generator function and the contextlib contextmanager decorator.

In [67]:
from contextlib import contextmanager


@contextmanager
def File_Context(fname):
    file = read_file(fname)
    columns = " ".join(next(file))
    data_NT = namedtuple((fname[0].upper() + fname[1:-4]), columns)
    try:
      yield data_NT(*next(file))
    finally:
      print("Data Extraction Finished")

In [71]:
File_Context('Cars.csv')

In [70]:
with File_Context('Cars.csv') as a:
  print(next(a))
  print(next(a))
  


Data Extraction Finished


TypeError: ignored

In [59]:
print(a)

Cars(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')


In [42]:
a

Cars(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')

In [43]:
a

Cars(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')